<a href="https://colab.research.google.com/github/cacomixtl/olist-ecommerce-data-analysis/blob/main/Olist_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import sqlite3
import pandas as pd

# 1. Define the path to your database file in Colab
# (This will be just the filename if you uploaded it directly to the root of session storage)
db_file_path = '/content/drive/MyDrive/Olist_Ecommerce/olist_project'

# 2. Create a database connection
try:
    conn = sqlite3.connect(db_file_path)
    print(f"Successfully connected to {db_file_path}")

    # Create a cursor object (optional for simple pd.read_sql_query)
    cur = conn.cursor()

    # 4. Write your SQL query
    # This is where you'll build the comprehensive query from Module 2
    # For now, let's test with a simple query:
    query = """
    -- Common Table Expression (CTE) to aggregate payment information per order
WITH OrderPaymentsAgg AS (
    SELECT
        op.order_id,
        SUM(op.payment_value) AS total_payment_value,
        -- Grouping payment types for an order.
        -- For SQLite, GROUP_CONCAT is available. For other SQL dialects, this might need adjustment (e.g., STRING_AGG).
        GROUP_CONCAT(DISTINCT op.payment_type) AS payment_types,
        SUM(CASE WHEN op.payment_type = 'credit_card' THEN op.payment_installments ELSE 0 END) AS total_credit_card_installments
    FROM order_payments op
    GROUP BY op.order_id
)
-- Main query to build the Analytical Base Table (ABT)
SELECT
    -- Order Information
    o.order_id,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,

    -- Order Item Information
    oi.order_item_id, -- Sequence of item in an order
    oi.product_id,
    oi.seller_id,
    oi.shipping_limit_date,
    oi.price AS item_price,
    oi.freight_value AS item_freight_value,

    -- Product Information
    p.product_category_name AS product_category_name_portuguese,
    pcnt.product_category_name_english,
    p.product_weight_g,
    p.product_length_cm,
    p.product_height_cm,
    p.product_width_cm,

    -- Customer Information
    c.customer_unique_id, -- Use this for customer-level analysis (RFM, etc.)
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,

    -- Seller Information
    s.seller_zip_code_prefix AS seller_zip_code_prefix,
    s.seller_city AS seller_city,
    s.seller_state AS seller_state,

    -- Aggregated Payment Information (from CTE)
    opa.total_payment_value,
    opa.payment_types,
    opa.total_credit_card_installments,

    -- Review Information
    r.review_score,
    r.review_comment_title, -- Included for potential qualitative insights later
    r.review_comment_message, -- Included for potential qualitative insights later
    r.review_creation_date,
    r.review_answer_timestamp

FROM orders o
INNER JOIN order_items oi
    ON o.order_id = oi.order_id
LEFT JOIN products p
    ON oi.product_id = p.product_id
LEFT JOIN product_name_translation pcnt
    ON p.product_category_name = pcnt.product_category_name
LEFT JOIN customers c
    ON o.customer_id = c.customer_id -- Note: o.customer_id links to a specific order's customer record
LEFT JOIN sellers s
    ON oi.seller_id = s.seller_id
LEFT JOIN OrderPaymentsAgg opa -- Joining the aggregated payment CTE
    ON o.order_id = opa.order_id
LEFT JOIN order_reviews r
    ON o.order_id = r.order_id     -- Assumes one main review per order for this join

-- Filtering for delivered orders, which are typically the focus of e-commerce analysis.
-- This can be removed or modified if other order statuses need to be analyzed.
WHERE
    o.order_status = 'delivered'
ORDER BY
    o.order_purchase_timestamp, -- Ordering by purchase time
    o.order_id,
    oi.order_item_id; -- Ensures a consistent order for the data

    """
    # Note: Adjust table and column names in the query above to EXACTLY match
    # how you defined them in your SQL SCHEMAS.sql and how they exist in your SQLite database.
    # For example, if your products table is named 'products', use 'products.product_id' etc.
    # I used 'prod' as an alias for 'products' in the join to avoid potential naming conflicts
    # if your actual table is 'products' and you also have a column named 'products' (unlikely but good practice).
    # Ensure your 'product_name_translation' table is correctly joined.

    # 5. Execute the query and load results into a Pandas DataFrame
    print("\nExecuting query...")
    df_main = pd.read_sql_query(query, conn)
    print("Query executed successfully.")

    # 6. Display the first few rows of the DataFrame
    print("\nFirst 5 rows of the resulting DataFrame:")
    print(df_main.head())

    print(f"\nShape of the DataFrame: {df_main.shape}")

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # 7. Close the connection
    if conn:
        conn.close()
        print("\nDatabase connection closed.")

Successfully connected to /content/drive/MyDrive/Olist_Ecommerce/olist_project

Executing query...
Query executed successfully.

First 5 rows of the resulting DataFrame:
                           order_id order_status order_purchase_timestamp  \
0  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
1  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
2  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
3  3b697a20d9e427646d92567910af6d57    delivered      2016-10-03 09:44:50   
4  be5bc2f0da14d8071e2d45451ad119d9    delivered      2016-10-03 16:56:50   

     order_approved_at order_delivered_carrier_date  \
0  2016-09-15 12:16:38          2016-11-07 17:11:53   
1  2016-09-15 12:16:38          2016-11-07 17:11:53   
2  2016-09-15 12:16:38          2016-11-07 17:11:53   
3  2016-10-06 15:50:54          2016-10-23 14:02:13   
4  2016-10-06 16:03:44          2016-10-21 16:33:46   

  order_delivered_customer_date orde

In [3]:
print("DataFrame Info:")
df_main.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110493 entries, 0 to 110492
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   order_id                          110493 non-null  object 
 1   order_status                      110493 non-null  object 
 2   order_purchase_timestamp          110493 non-null  object 
 3   order_approved_at                 110493 non-null  object 
 4   order_delivered_carrier_date      110493 non-null  object 
 5   order_delivered_customer_date     110493 non-null  object 
 6   order_estimated_delivery_date     110493 non-null  object 
 7   order_item_id                     110493 non-null  int64  
 8   product_id                        110493 non-null  object 
 9   seller_id                         110493 non-null  object 
 10  shipping_limit_date               110493 non-null  object 
 11  item_price                        11

Data type correction

In [4]:
print("Converting timestamp columns to datetime...")
timestamp_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'shipping_limit_date',
    'review_creation_date', 'review_answer_timestamp'
]
for col in timestamp_cols:
    df_main[col] = pd.to_datetime(df_main[col], errors='coerce')
print("Timestamp conversion done.")
# Verify:
# print(df_main[timestamp_cols].info())
# print(df_main[timestamp_cols].isnull().sum()) # Check for new NaTs


Converting timestamp columns to datetime...
Timestamp conversion done.


In [5]:
print("\nConverting product dimension columns to numeric...")
product_dimension_cols = [
    'product_weight_g', 'product_length_cm',
    'product_height_cm', 'product_width_cm'
]
for col in product_dimension_cols:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')
    # Consider converting to integer if appropriate after checking for decimals
    # For now, float64 (default for to_numeric) is fine.
print("Product dimension conversion done.")
# Verify:
# print(df_main[product_dimension_cols].info())
# print(df_main[product_dimension_cols].isnull().sum()) # Check for NaNs if conversion failed for some rows



Converting product dimension columns to numeric...
Product dimension conversion done.


In [6]:
print("\nConverting zip code columns to string...")
zip_code_cols = ['customer_zip_code_prefix', 'seller_zip_code_prefix']
for col in zip_code_cols:
    df_main[col] = df_main[col].astype(str)
print("Zip code conversion done.")
# Verify:
# print(df_main[zip_code_cols].info())



Converting zip code columns to string...
Zip code conversion done.


In [7]:
print(f"\nShape before dropping payment NaNs: {df_main.shape}")
df_main.dropna(subset=['total_payment_value'], inplace=True) # This will drop rows where any of these are NaN if they are the same 3 rows
print(f"Shape after dropping payment NaNs: {df_main.shape}")



Shape before dropping payment NaNs: (110493, 34)
Shape after dropping payment NaNs: (110490, 34)


In [10]:
print("\nConverting 'total_credit_card_installments' to integer...")
df_main['total_credit_card_installments'] = df_main['total_credit_card_installments'].astype('int64')
print("'total_credit_card_installments' conversion done.")
# Verify:
#print(df_main[['total_credit_card_installments']].info())



Converting 'total_credit_card_installments' to integer...
'total_credit_card_installments' conversion done.


Data imputation

In [14]:
print("\nImputing missing 'product_category_name_english'...")
df_main['product_category_name_english'].fillna('Unknown Category', inplace=True)
print("'product_category_name_english' imputation done.")
# Verify:
print(df_main['product_category_name_english'].isnull().sum())



Imputing missing 'product_category_name_english'...
'product_category_name_english' imputation done.
0


In [15]:
print("\nImputing missing review text fields...")
df_main['review_comment_title'].fillna('No Title Provided', inplace=True)
df_main['review_comment_message'].fillna('No Comment Provided', inplace=True)
print("Review text fields imputation done.")
# Verify:
print(df_main[['review_comment_title', 'review_comment_message']].isnull().sum())



Imputing missing review text fields...
Review text fields imputation done.
review_comment_title      0
review_comment_message    0
dtype: int64


Duplication check

In [16]:
print(f"\nNumber of full duplicate rows: {df_main.duplicated().sum()}")
if df_main.duplicated().sum() > 0:
    df_main.drop_duplicates(inplace=True)
    print(f"Dropped duplicate rows. New shape: {df_main.shape}")



Number of full duplicate rows: 0


Data type validation


In [17]:
print("DataFrame Info:")
df_main.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 110490 entries, 3 to 110492
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   order_id                          110490 non-null  object        
 1   order_status                      110490 non-null  object        
 2   order_purchase_timestamp          110490 non-null  datetime64[ns]
 3   order_approved_at                 110475 non-null  datetime64[ns]
 4   order_delivered_carrier_date      110488 non-null  datetime64[ns]
 5   order_delivered_customer_date     110482 non-null  datetime64[ns]
 6   order_estimated_delivery_date     110490 non-null  datetime64[ns]
 7   order_item_id                     110490 non-null  int64         
 8   product_id                        110490 non-null  object        
 9   seller_id                         110490 non-null  object        
 10  shipping_limit_date  

In [18]:
print("\nDescriptive Statistics (Numerical):")
print(df_main.describe())


Descriptive Statistics (Numerical):
            order_purchase_timestamp              order_approved_at  \
count                         110490                         110475   
mean   2018-01-01 22:49:30.717060608  2018-01-02 10:23:17.652138240   
min              2016-10-03 09:44:50            2016-10-04 09:43:32   
25%    2017-09-14 21:55:54.249999872            2017-09-15 11:30:03   
50%       2018-01-20 21:21:11.500000            2018-01-22 13:52:06   
75%              2018-05-05 11:09:04     2018-05-05 17:55:13.500000   
max              2018-08-29 15:00:37            2018-08-29 15:10:26   
std                              NaN                            NaN   

        order_delivered_carrier_date  order_delivered_customer_date  \
count                         110488                         110482   
mean   2018-01-05 05:41:37.502316800  2018-01-14 10:01:45.804936704   
min              2016-10-08 10:34:01            2016-10-11 13:46:32   
25%       2017-09-18 21:27:25.500000   

In [ ]:
print("\nDescriptive Statistics (Object/Categorical):")
print(df_main.describe(include=['object', 'category']))


Descriptive Statistics (Object/Categorical):
                                order_id order_status  \
count                             110493       110493   
unique                             96478            1   
top     5a3b1c29a49756e75f1ef513383c0c12    delivered   
freq                                  22       110493   

       order_purchase_timestamp    order_approved_at  \
count                    110493               110493   
unique                    95956                88275   
top         2017-10-17 13:06:29  2018-02-24 03:20:27   
freq                         22                   23   

       order_delivered_carrier_date order_delivered_customer_date  \
count                        110493                        110493   
unique                        80107                         95659   
top             2018-05-09 15:48:00           2017-10-22 14:43:54   
freq                             48                            22   

       order_estimated_delivery_date     

In [ ]:
print("\nMissing Values per Column:")
print(df_main.isnull().sum())

print("\nPercentage of Missing Values per Column:")
print((df_main.isnull().sum() / len(df_main)) * 100)



Missing Values per Column:
order_id                               0
order_status                           0
order_purchase_timestamp               0
order_approved_at                      0
order_delivered_carrier_date           0
order_delivered_customer_date          0
order_estimated_delivery_date          0
order_item_id                          0
product_id                             0
seller_id                              0
shipping_limit_date                    0
item_price                             0
item_freight_value                     0
product_category_name_portuguese       0
product_category_name_english       1565
product_weight_g                       0
product_length_cm                      0
product_height_cm                      0
product_width_cm                       0
customer_unique_id                     0
customer_zip_code_prefix               0
customer_city                          0
customer_state                         0
seller_zip_code_prefix       

In [ ]:
print("\nFirst few rows:")
print(df_main.head())

print("\nLast few rows:")
print(df_main.tail())

print("\nRandom sample of rows:")
print(df_main.sample(5))


First few rows:
                           order_id order_status order_purchase_timestamp  \
0  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
1  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
2  bfbd0f9bdef84302105ad712db648a6c    delivered      2016-09-15 12:16:38   
3  3b697a20d9e427646d92567910af6d57    delivered      2016-10-03 09:44:50   
4  be5bc2f0da14d8071e2d45451ad119d9    delivered      2016-10-03 16:56:50   

     order_approved_at order_delivered_carrier_date  \
0  2016-09-15 12:16:38          2016-11-07 17:11:53   
1  2016-09-15 12:16:38          2016-11-07 17:11:53   
2  2016-09-15 12:16:38          2016-11-07 17:11:53   
3  2016-10-06 15:50:54          2016-10-23 14:02:13   
4  2016-10-06 16:03:44          2016-10-21 16:33:46   

  order_delivered_customer_date order_estimated_delivery_date  order_item_id  \
0           2016-11-09 07:47:38           2016-10-04 00:00:00              1   
1           2016-11-09 07:47:

The following cell will drop all the rows with missing values.

In [ ]:
df_main_cleaned = df_main.dropna(subset=[
    'product_category_name_english',
    'total_payment_value',
    'payment_types',
    'total_credit_card_installments',
    'review_score',
    'review_comment_title',
    'review_comment_message',
    'review_creation_date',
    'review_answer_timestamp'], inplace=False)

Let's see how it changes

In [ ]:
print("\nMissing Values per Column:")
print(df_main_cleaned.isnull().sum())

print("\nPercentage of Missing Values per Column:")
print((df_main_cleaned.isnull().sum() / len(df_main)) * 100)


Missing Values per Column:
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_carrier_date        0
order_delivered_customer_date       0
order_estimated_delivery_date       0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
item_price                          0
item_freight_value                  0
product_category_name_portuguese    0
product_category_name_english       0
product_weight_g                    0
product_length_cm                   0
product_height_cm                   0
product_width_cm                    0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
seller_zip_code_prefix              0
seller_city                         0
seller_state          

Correcting the data type on intended timestamps

In [ ]:
timestamp_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'shipping_limit_date',
    'review_creation_date', 'review_answer_timestamp'
]
for col in timestamp_cols:
    df_main_cleaned[col] = pd.to_datetime(df_main_cleaned[col], errors='coerce') # errors='coerce' will turn unparseable dates into NaT (Not a Time)


<ipython-input-23-656be8b0f0b7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_cleaned[col] = pd.to_datetime(df_main_cleaned[col], errors='coerce') # errors='coerce' will turn unparseable dates into NaT (Not a Time)
<ipython-input-23-656be8b0f0b7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_cleaned[col] = pd.to_datetime(df_main_cleaned[col], errors='coerce') # errors='coerce' will turn unparseable dates into NaT (Not a Time)
<ipython-input-23-656be8b0f0b7>:8: SettingWithCopyWarning: 
